Goal 1:

Refactor the `Polygon` class so that all the calculated properties are lazy properties, i.e. they should still be calculated properties, but they should not have to get recalculated more than once (since we made our `Polygon` class "immutable").
 

Goal 2:


Refactor the `Polygons` (sequence) type, into an **iterable**. Make sure also that the elements in the iterator are computed lazily - i.e. you can no longer use a list as an underlying storage mechanism for your polygons.

You'll need to implement both an iterable and an iterator.

In [78]:
import math

class Polygon2:
    def __init__(self, n, R):
        if n < 3:
            raise ValueError('Polygon must have at least 3 vertices.')
        self.edge = n  #Edges
        self.radius = R  #Circum-Radius
        
    def __repr__(self):
        return f'Polygon(n={self._n}, R={self._R})'
    
    @property
    def edge(self):
        return self._n
    
    @edge.setter
    def edge(self, n):
        self._n = n
        self._int_angle = None
        self._side_len  = None
        self._apothem   = None
        self._area      = None
        self._perimeter = None
  
    @property
    def radius(self):
        return self._R
    
    @radius.setter
    def radius(self, R):
        self._R = R
        self._side_len  = None
        self._apothem   = None
        self._area      = None
        self._perimeter = None
    
    @property
    def interior_angle(self):
        if self._int_angle is None:
            print('Calc IA')
            self._int_angle = (self._n - 2) * 180 / self._n
        return self._int_angle    
    
    @property
    def side_length(self):
        if self._side_len is None:
            print('calc SL')
            self._side_len = 2 * self._R * math.sin(math.pi / self._n)
        return self._side_len
    
    @property
    def apothem(self):
        if self._apothem is None:
            print('calc apothem')
            self._apothem = self._R * math.cos(math.pi / self._n)
        return self._apothem
    
    @property
    def area(self):
        if self._area is None:
            print('calc area')
            self._area = self._n / 2 * self.side_length * self.apothem
        return self._area
    
    @property
    def perimeter(self):
        if self._perimeter is None:
            print('calc perimeter')
            self._perimeter = self._n * self.side_length
        return self._perimeter
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (self._n == other._n 
                    and self._R == other._R)
        else:
            return NotImplemented
        
    def __gt__(self, other):
        if isinstance(other, self.__class__):
            return self._n > other._n
        else:
            return NotImplemented

In [79]:
p = Polygon2(4, 10)

In [80]:
# Calculating IA, sl, a, p
print('p.edge', p.edge)
print('p.radius', p.radius)
print('p.interior_angle', p.interior_angle)
print('p.side_length', p.side_length)
print('p.apothem', p.apothem)
print('p.area', p.area)
print('p.perimeter', p.perimeter)

p.edge 4
p.radius 10
Calc IA
p.interior_angle 90.0
calc SL
p.side_length 14.142135623730951
calc apothem
p.apothem 7.0710678118654755
calc area
p.area 200.00000000000003
calc perimeter
p.perimeter 56.568542494923804


In [81]:
# It should give from cache. No calc statements should appear
print('p.edge', p.edge)
print('p.radius', p.radius)
print('p.interior_angle', p.interior_angle)
print('p.side_length', p.side_length)
print('p.apothem', p.apothem)
print('p.area', p.area)
print('p.perimeter', p.perimeter)

p.edge 4
p.radius 10
p.interior_angle 90.0
p.side_length 14.142135623730951
p.apothem 7.0710678118654755
p.area 200.00000000000003
p.perimeter 56.568542494923804


In [82]:
# Setting edge to a diff. value
p.edge = 5

In [83]:
#Everything should get re-calculated bcoz edge changed
print('p.edge', p.edge)
print('p.radius', p.radius)
print('p.interior_angle', p.interior_angle)
print('p.side_length', p.side_length)
print('p.apothem', p.apothem)
print('p.area', p.area)
print('p.perimeter', p.perimeter)

p.edge 5
p.radius 10
Calc IA
p.interior_angle 108.0
calc SL
p.side_length 11.755705045849464
calc apothem
p.apothem 8.090169943749475
calc area
p.area 237.76412907378844
calc perimeter
p.perimeter 58.77852522924732


In [84]:
#Setting radius to a diff. value
p.radius = 11

In [85]:
##Everything should get re-calculated bcoz radius changed except IA. radius has no relation with IA.
print('p.edge', p.edge)
print('p.radius', p.radius)
print('p.interior_angle', p.interior_angle)
print('p.side_length', p.side_length)
print('p.apothem', p.apothem)
print('p.area', p.area)
print('p.perimeter', p.perimeter)

p.edge 5
p.radius 11
p.interior_angle 108.0
calc SL
p.side_length 12.931275550434409
calc apothem
p.apothem 8.899186938124423
calc area
p.area 287.694596179284
calc perimeter
p.perimeter 64.65637775217205


In [86]:
p

Polygon(n=5, R=11)

In [87]:
p1 = Polygon2(5, 11)
p2 = Polygon2(6, 11)
p3 = Polygon2(4, 11)

In [88]:
print(p == p1) # True
print(p > p1) # False
print(p > p2) # False
print(p2 > p) # True
print(p > p3) # True

True
False
False
True
True


In [100]:
class Polygons2:
    def __init__(self, m, R):
        if m < 3:
            raise ValueError('m must be greater than 3')
        self._m = m
        self._R = R
        self._polygons = None
        
    def __len__(self):
        return self._m - 2  # To exclude sides=1 and sides=2 as they are not polygons
    
    def __repr__(self):
        return f'Polygons(m={self._m}, R={self._R})'
    
    def __iter__(self):
        return self.polyiterator(self)
    
    #def __getitem__(self, s):
    #    return self._polygons[s]
    
    @property
    def max_efficiency_polygon(self):
        if self._polygons is None:
            print('calc ME')
            self._polygons   = [Polygon2(i, self._R) for i in range(3, self._m+1)]
            self._polygons   = sorted(self._polygons, 
                                      key=lambda p: p.area/p.perimeter,
                                      reverse=True)            
        return self._polygons[0]
    
    class polyiterator:
        def __init__(self, poly_obj):
            self._index = 3
            self.poly_obj = poly_obj
            
        def __iter__(self):
            return self
        
        def __next__(self):
            if self._index > self.poly_obj._m:
                raise StopIteration
            else:
                print(f'calc poly, {self._index} out of {self.poly_obj._m}')
                poly_ =  Polygon2(self._index, self.poly_obj._R)
                result =  f'Sides = {poly_.edge}, Radius = {poly_.radius}, Eff_ratio = {poly_.area/poly_.perimeter}' 
                self._index += 1
                return result

In [101]:
pc = Polygons2(6, 10)

In [102]:
# Everything should get calculated
pc.max_efficiency_polygon

calc ME
calc area
calc SL
calc apothem
calc perimeter
calc area
calc SL
calc apothem
calc perimeter
calc area
calc SL
calc apothem
calc perimeter
calc area
calc SL
calc apothem
calc perimeter


Polygon(n=6, R=10)

In [103]:
# This time it should take from cache
pc.max_efficiency_polygon

Polygon(n=6, R=10)

In [104]:
for i in pc:
    print(i)

calc poly, 3 out of 6
calc area
calc SL
calc apothem
calc perimeter
Sides = 3, Radius = 10, Eff_ratio = 2.5000000000000004
calc poly, 4 out of 6
calc area
calc SL
calc apothem
calc perimeter
Sides = 4, Radius = 10, Eff_ratio = 3.535533905932738
calc poly, 5 out of 6
calc area
calc SL
calc apothem
calc perimeter
Sides = 5, Radius = 10, Eff_ratio = 4.045084971874737
calc poly, 6 out of 6
calc area
calc SL
calc apothem
calc perimeter
Sides = 6, Radius = 10, Eff_ratio = 4.330127018922194


In [105]:
iter1 = iter(pc)

In [106]:
next(iter1)

calc poly, 3 out of 6
calc area
calc SL
calc apothem
calc perimeter


'Sides = 3, Radius = 10, Eff_ratio = 2.5000000000000004'

In [107]:
next(iter1)

calc poly, 4 out of 6
calc area
calc SL
calc apothem
calc perimeter


'Sides = 4, Radius = 10, Eff_ratio = 3.535533905932738'

In [108]:
next(iter1)

calc poly, 5 out of 6
calc area
calc SL
calc apothem
calc perimeter


'Sides = 5, Radius = 10, Eff_ratio = 4.045084971874737'

In [109]:
next(iter1)

calc poly, 6 out of 6
calc area
calc SL
calc apothem
calc perimeter


'Sides = 6, Radius = 10, Eff_ratio = 4.330127018922194'

In [99]:
next(iter1)

StopIteration: 